# Non-linear Registration
As we saw in the last section, affine transformations alone are not enough to accurately register individual subjects to a common template space. In order to move beyond simple affine approaches, we have to consider methods where we can move each voxel *independently*. These are known as *non-linear* registration techniques. Generally, these methods are more complex than affine methods and so we will keep the detail presented here limited. However, much of the image registration theory remains relevant. We still have a *reference* image (the MNI templates) that we want to align our *source* image to. In order to do so, we still have to estimate some form of *mapping* between the source and the reference images, and this will require optimisation using some form of *objective function*. We then need to *resample* the source image to move it into standard space. The main difference is that our mapping is no longer given by a 12-parameter affine transformation. Instead, our mapping is based on a *deformation field*.​​​

## Deformation Fields
At its most basic, a deformation field is an image where each voxel value indicates how to move that specific voxel when warping to standard space. There are therefore three of these images, one for the $x$-axis, $y$-axis and $z$-axis. In `SPM`, the deformation field images contain the new millimetre coordinates for each voxel after warping. So the $X$ deformation field contains all the new $x$-coordinates, the $Y$ deformation field contains all the new $y$-coordinates and the $Z$ deformation field contains all the new $z$-coordinates. So rather than using the header matrices to calculate the millimetre coordinates, we can do it directly by looking at the deformation fields. We can then convert these into voxel coordinates in the source image to give us the values we need to copy into our new normalised image. Of course, given how flexible the non-linear approach is, this will almost always need some form of interpolation. 

An examples of a deformation field for the $x$-axis is shown below. As you navigate around the image, compare the $x$-coordinate in red to the voxel value in white. The $x$-coordinate gives the original world-space location of that voxel and the voxel value in white gives the *new* world-space coordinate where the voxel must be moved in order to warp the image to standard space. For instance, if the coordinates of the voxel are given as $[-18, 12, 0]$ and the voxel value is $-20.01$, this means that the voxel located at $[-18, 12, 0]$ must be shifted to a world-space $x$-coordinate of $-20.01$. The deformation field for the $y$-axis and $z$-axis would then be used in the same fashion to complete the new world-space coordinates.

<iframe src="deform-x.html" width="800" height="600" frameborder="0" scrolling="no" title=""></iframe>

```{Important}
Because the deformation field defines a warping in world-space, it can be applied to *any* image that is registered with the source image. This will become important when we get get to fMRI pre-processing, as it is typical to use an aligned anatomical scan to estimate the warping, before appplying the resultant deformation field to the functional images. As such, a deformation field is *reuseable*, so long as the images are aligned in world-space.
```

### Creating Deformation Fields
The first step in creating a deformation field is to align the source and the target using an affine approach, in order to get the images as close as possible. So even when performing non-linear registration, the algorithm always starts by doing the best job it can with an affine transformation. As such, everything said earlier in this lesson about making sure the linear registration performs well also applies here. Issues with optimisation also remain the same, so making sure the images are as well-aligned as possible beforehand is always a good idea.

Next, an optimisation algorithm is used to estimate the warping needed to match the images. However, unlike an affine transformation where there are (at most) 12 parameters, non-linear registration can have *thousands* of parameters to optimise. How this is done is somewhat beyond the scope of the lesson, however, we can gain some intuition by looking at a deformation field in 1D. In {numref}`deform-1d-fig` we can see a plot of a single row (shown in blue) of the $X$ deformation field. A slice has been chosen where all the voxels originally has an x-coordinate of 0mm. The orange line shows the new millimetre coordinates for all voxels from this row, indicating that they will all be shifted by different amounts to the right.

```{figure} images/deform-1d.png 
width: 800px
name: deform-1d-fig
---
Illustration of a deformation field in 1D.
```
This orange line is what the non-linear registration estimates. It does this by taking a whole bunch of cosines of different frequencies, weighting each frequency and then summing them to create these lines. The optimisation tries to find the weights (known as the *coefficients*) that lead to a line that represent the best shift in the image coordinates to fit the template image. As with linear registration, this is about finding the weights that minimise a cost function, which for non-linear registration is usually *least-squares*. As such, it is typical to have multiple MNI templates available, such as a T1, T2 and EPI for normalising images from different modalities.

## Regularisation

An important element of non-linear techniques is making sure that the degree of warping applied to the images is sensible. What we do not want is to end up with biologically implausible brains that have been severely warped to suit the whims of the optimisation algorithm. As such, non-linear algorithms usually come with a way of penalising too much warping. This is known as regularisation. In SPM, this is done using the bending energy of the warp. The more extreme the bending, the larger the difference it has to make to the cost function to be allowed. In practise, this means that biologically implausible warps should be penalised and not allowed to happen. The consequences of reducing the amount of regularisation is shown below.